#### In this project, I worked on a Kaggle fake news dataset using both classical machine learning methods (TF-IDF with Logistic Regression and Naive Bayes) and deep learning (BERT). Throughout the process, I focused on data preprocessing, feature extraction, modeling, and evaluation. While the results did not reach high accuracy levels (~50%), the experience provided valuable insights into the challenges of working with real-world text data — such as limited vocabulary diversity, label reliability, and high contextual similarity between classes. This notebook summarizes each step taken, the observations made, and the key takeaways from this learning-focused project. It was a great opportunity to gain hands-on experience in data analysis, machine learning pipelines, and model diagnostics.

In [ ]:
# Gerekli kütüphaneleri yükleme
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import transformers

# NLTK veri setlerini indir
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# Veri setini yükle
df = pd.read_csv('/kaggle/input/fake-news-detection-dataset/fake_news_dataset.csv') 
print(df.head())
print(df.info())
print(df['label'].value_counts())
print(df[['text', 'label']].isna().sum())

In [ ]:
# Stop kelimeleri yükle, negatif kelimeleri koru
stop_words = set(stopwords.words('english')) - {'not', 'no', 'nor'}
lemmatizer = WordNetLemmatizer()

In [ ]:
# Ön işleme fonksiyonu
def preprocess_text(text):
    if isinstance(text, str):
        tokens = word_tokenize(text.lower())  # Küçük harfe çevir ve tokenize et
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
        return ' '.join(tokens)
    return ''

In [ ]:
# Metin sütununa uygula
df['clean_text'] = df['text'].apply(preprocess_text)

In [ ]:
# Eksik verileri kontrol et ve temizle
df = df[df['clean_text'] != '']  # Boş metinleri kaldır
print(df['clean_text'].head())
print(df.shape)
print(df['clean_text'].apply(lambda x: len(x.split())).describe())

In [ ]:
# TF-IDF vektörleştirme
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(df['clean_text'])
y = df['label'].map({'fake': 0, 'real': 1})  # Etiketleri sayısal yap: fake=0, real=1

In [ ]:
# Train/test böl
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)

In [ ]:
# Logistic Regression modeli
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

In [ ]:
# Tahmin ve değerlendirme
y_pred = lr_model.predict(X_test)
print('Logistic Regression Accuracy:', accuracy_score(y_test, y_pred))
print('Logistic Regression F1-Score:', f1_score(y_test, y_pred))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Logistic Regression Confusion Matrix')
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.show()

In [ ]:
from datasets import Dataset
from transformers import BertTokenizer

In [ ]:
# Pandas’tan Hugging Face Dataset’e çevir
data_dict = {'text': df['text'], 'labels': df['label'].map({'fake': 0, 'real': 1})}
dataset = Dataset.from_dict(data_dict)

In [ ]:
# Train/test böl
train_test = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test['train']
test_dataset = train_test['test']

In [ ]:
# BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Tokenize fonksiyonu
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

In [ ]:
# Veriyi tokenize et
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

In [ ]:
# Dataset formatını ayarla
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
# Etiketleri kontrol et
print(train_dataset[0]['labels'])  # 0 veya 1 (integer) olmalı
print(type(train_dataset[0]['labels']))  # <class 'torch.Tensor'> olmalı

In [ ]:
# Modeli yükle
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

In [ ]:
# Eğitim parametreleri
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=100,
    report_to='none',  
)

In [ ]:
# Metrik hesaplama fonksiyonu
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {'accuracy': acc, 'f1': f1}

In [ ]:
# Trainer oluştur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Modeli eğit
trainer.train()

In [ ]:
# Gerekli kütüphaneler
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Veri setini yükle
df = pd.read_csv('/kaggle/input/fake-news-detection-dataset/fake_news_dataset.csv')

In [ ]:
# Sahte ve gerçek metinler
fake_text = ' '.join(df[df['label'] == 'fake']['text'])
real_text = ' '.join(df[df['label'] == 'real']['text'])

In [ ]:
# Sahte haberler kelime bulutu
fake_wc = WordCloud(width=800, height=400, max_words=100).generate(fake_text)
plt.figure(figsize=(10, 5))
plt.imshow(fake_wc, interpolation='bilinear')
plt.title('Sahte Haberler Kelime Bulutu')
plt.axis('off')
plt.show()

In [ ]:
# Gerçek haberler kelime bulutu
real_wc = WordCloud(width=800, height=400, max_words=100).generate(real_text)
plt.figure(figsize=(10, 5))
plt.imshow(real_wc, interpolation='bilinear')
plt.title('Gerçek Haberler Kelime Bulutu')
plt.axis('off')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Sahte ve gerçek metinler
fake_texts = df[df['label'] == 'fake']['text']
real_texts = df[df['label'] == 'real']['text']

In [ ]:
# CountVectorizer ile kelime frekanslarını hesapla
vectorizer = CountVectorizer(stop_words='english', max_features=20)

In [ ]:
# Sahte haberler için
fake_vectorizer = vectorizer.fit(fake_texts)
fake_freq = fake_vectorizer.transform(fake_texts)
fake_words = fake_vectorizer.get_feature_names_out()
fake_counts = fake_freq.sum(axis=0).A1
fake_word_freq = dict(zip(fake_words, fake_counts))

In [ ]:
# Gerçek haberler için
real_vectorizer = vectorizer.fit(real_texts)
real_freq = real_vectorizer.transform(real_texts)
real_words = real_vectorizer.get_feature_names_out()
real_counts = real_freq.sum(axis=0).A1
real_word_freq = dict(zip(real_words, real_counts))

In [ ]:
# Sonuçları yazdır
print("Sahte Haberlerde En Sık 20 Kelime:")
for word, count in sorted(fake_word_freq.items(), key=lambda x: x[1], reverse=True):
    print(f"{word}: {count}")

print("\nGerçek Haberlerde En Sık 20 Kelime:")
for word, count in sorted(real_word_freq.items(), key=lambda x: x[1], reverse=True):
    print(f"{word}: {count}")

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np

In [ ]:
# Kısa metinleri çıkar
df = df[df['text'].str.split().str.len() > 50]

In [ ]:
# TF-IDF ve basit model
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['text'])
y = df['label'].map({'fake': 0, 'real': 1})

In [ ]:
# Logistic Regression ile güvenilirlik tahmini
model = LogisticRegression(max_iter=1000)
model.fit(X, y)
probs = model.predict_proba(X)

In [ ]:
# Düşük güvenilirlikli örnekleri çıkar (olasılık 0.4-0.6 arası)
mask = ~((probs[:, 0] > 0.4) & (probs[:, 0] < 0.6))
df_clean = df[mask]

print("Temizlenmiş veri boyutu:", df_clean.shape)

In [ ]:
from textblob import TextBlob
import pandas as pd

In [ ]:
# Duygusal ton hesapla
df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [ ]:
# Sahte ve gerçek haberlerin ortalama sentimentini karşılaştır
print("Sahte Haberler Ortalama Sentiment:", df[df['label'] == 'fake']['sentiment'].mean())
print("Gerçek Haberler Ortalama Sentiment:", df[df['label'] == 'real']['sentiment'].mean())

In [ ]:
# Sahte ve gerçek haberlerden rastgele 3 örnek
print("Sahte Haber Örnekleri:")
for text in df[df['label'] == 'fake']['text'].sample(3, random_state=42):
    print(f"- {text[:200]}...\n")

print("Gerçek Haber Örnekleri:")
for text in df[df['label'] == 'real']['text'].sample(3, random_state=42):
    print(f"- {text[:200]}...\n")

In [ ]:
# Özet
print("Veri Seti Özeti:")
print(f"- Toplam örnek: {len(df)}")
print(f"- Temizlenmiş örnek: 5721")
print(f"- Sahte haber sentiment: 0.094")
print(f"- Gerçek haber sentiment: 0.093")
print("- Sorun: Gürültülü etiketler ve bağlamsal benzerlik. Kelime bulutlarında 'new', 'mr' gibi ortak kelimeler, BERT %48 accuracy.")

# Sahte Haber Tespiti Projesi: Özet ve Değerlendirme

### Bu projede, Kaggle’daki bir sahte haber veri setiyle sahte ve gerçek haberleri sınıflandırmaya çalıştık. Amacımız, makine öğrenmesi (TF-IDF) ve derin öğrenme (BERT) modelleriyle yüksek doğruluk elde etmekti. Ancak veri setinin kalitesi nedeniyle beklenen sonuçları alamadık. Aşağıda, yaptığımız adımları, sonuçları ve öğrendiklerimizi özetledik.

## 1. Veri Seti Yükleme ve İnceleme
- **Ne Yaptık?**: `/kaggle/input/fake-news-detection-dataset/fake_news.csv` veri setini yükledik. 20,000 örnekten oluşan veri setinde `text` (haber metni) ve `label` (sahte/gerçek) sütunları vardı.
- **Neden Yaptık?**: Veri setinin yapısını anlamak ve sahte/gerçek haberlerin dağılımını kontrol etmek için.
- **Sonuç**: Veri seti dengeliydi (yaklaşık 10,000 sahte, 10,000 gerçek), ama metinlerin kalitesi şüpheliydi.

## 2. TF-IDF ile Makine Öğrenmesi
- **Ne Yaptık?**: Metinleri TF-IDF ile sayısal vektörlere çevirdik (`max_features=10000`), Logistic Regression, Naive Bayes gibi modeller denedik.
- **Neden Yaptık?**: Basit makine öğrenmesi modelleriyle hızlı sonuç almak için.
- **Sonuç**: TF-IDF sadece 870 özellik üretti, modeller ~%50 doğruluk verdi. Bu, veri setindeki kelime dağarcığının sınırlı olduğunu gösterdi.

## 3. BERT ile Derin Öğrenme
- **Ne Yaptık?**: BERT (`bert-base-uncased`) modelini 16,000 örnekle 3 epoch eğittik (batch_size=8, Kaggle P100 GPU). Eğitim parametreleri: `TrainingArguments` ile `eval_strategy='epoch'`, `report_to='none'`.
- **Neden Yaptık?**: BERT, metinlerin bağlamını yakalayarak daha iyi sonuç verebilirdi.
- **Sonuç**: 9 saat süren eğitim sonrası %48 doğruluk ve ~0.65 F1-score aldık. Model rastgele tahmin seviyesindeydi, veri setinin ayrım yapmaya uygun olmadığını gösterdi.
- **Sorunlar**:
  - `NameError: TrainingArguments`, `wandb.init()` gibi hatalar çıktı, `report_to='none'` ile çözdük.
  - Checkpoint’ler (`./results/checkpoint-6000`) Kaggle’da kayboldu, `HFValidationError` aldık.

## 4. Veri Seti Analizi
- **Ne Yaptık?**:
  - **Kelime Bulutları**: Sahte ve gerçek haberlerin kelime bulutlarını oluşturduk. `new`, `mr` gibi ortak kelimeler çıktı.
  - **Kelime Frekansları**: `CountVectorizer` ile en sık 20 kelimeyi listeledik (sahte: `summer`, `wide`; gerçek: `economy`, `member`). Kelimeler farklıydı, ama bağlamsal ayrım için yetersizdi.
  - **Sentiment Analizi**: `TextBlob` ile duygusal ton hesapladık (sahte: 0.094, gerçek: 0.093). Skorlar çok yakındı, ayrım yapılamadı.
  - **Veri Temizleme**: Düşük güvenilirlikli örnekleri çıkardık, veri 5,721’e düştü.
- **Neden Yaptık?**: Veri setinin neden başarısız olduğunu anlamak için.
- **Sonuç**: Veri setinde gürültülü etiketler ve bağlamsal benzerlik vardı. Metinler, sahte/gerçek ayrımı için yeterince farklı değildi.

## 5. Örnek Metinler
- **Sahte Haber Örnekleri**:
  - `where free section small present stage couple memory bag would real protect page notice...`
  - `or onto strategy first camera stage really almost beautiful whole land thus care...`
  - `executive reflect this family hard drive summer author direction source option help...`
- **Gerçek Haber Örnekleri**:
  - `ready best rich computer choose middle center expert several rich of nearly voice...`
  - `free themselves keep bill final inside all federal popular serious claim nearly...`
  - `seat reach TV sometimes population treatment ability until outside card case several...`
- **Yorum**: Metinler benzer kelimeler içeriyor, ayrım yapmak zor.

## 6. Öğrendiklerimiz
- **Veri Kalitesi**: Sahte haber tespiti için veri setinin etiket kalitesi ve metinlerin ayırt ediciliği kritik.
- **Model Sınırları**: TF-IDF ve BERT, kaliteli veri olmadan iyi sonuç vermiyor.
- **Analiz Teknikleri**: Kelime bulutları, frekans analizi ve sentiment analizi veri seti teşhisinde etkili.
- **Kaggle Deneyimi**: Uzun süren eğitimlerde checkpoint’leri kaydetme ve Kernel zaman aşımı sorunlarını öğrendik.

## 7. Neden Başarısız Olduk?
- **Gürültülü Etiketler**: Sahte/gerçek etiketleri yanlış veya tutarsız olabilir.
- **Bağlamsal Benzerlik**: Metinler aynı konuları benzer kelimelerle anlatıyor (ör. `new`, `mr`).
- **Düşük Kelime Çeşitliliği**: TF-IDF sadece 870 özellik üretti, veri seti sınırlı.

## 8. Sonuç ve Kapanış
- Bu veri setiyle sahte haber tespiti yapmak mümkün olmadı. 9 saatlik BERT denemesi, TF-IDF modelleri ve veri analizleri veri setinin yetersiz olduğunu gösterdi.
- **Karar**: Veri setini kapatıyoruz. Yeni bir veri seti (ör. LIAR) ile denemek daha mantıklı olurdu, ama bu projeyi burada sonlandırıyoruz.
- **Teşekkür**: Bu süreçte veri analizi, makine öğrenmesi ve derin öğrenme konusunda büyük tecrübe kazandım.


# Fake News Detection Project: Summary and Evaluation

### In this project, we attempted to classify fake and real news using a dataset from Kaggle. Our goal was to achieve high accuracy using machine learning (TF-IDF) and deep learning (BERT) models. However, due to the dataset's quality, we couldn’t obtain satisfactory results. Below, we summarize the steps, outcomes, and key takeaways.

## 1. Data Loading and Exploration
- **What We Did**: Loaded the dataset from `/kaggle/input/fake-news-detection-dataset/fake_news.csv`. It contained 20,000 samples with `text` (news content) and `label` (fake/real) columns.
- **Why**: To understand the dataset structure and check the distribution of fake/real news.
- **Result**: The dataset was balanced (~10,000 fake, 10,000 real), but the text quality raised concerns.

## 2. TF-IDF with Machine Learning
- **What We Did**: Converted texts to numerical vectors using TF-IDF (`max_features=10000`) and tested models like Logistic Regression and Naive Bayes.
- **Why**: To obtain quick results with simple machine learning models.
- **Result**: TF-IDF produced only 870 features, and models achieved ~50% accuracy, indicating limited vocabulary diversity in the dataset.

## 3. BERT with Deep Learning
- **What We Did**: Trained a BERT model (`bert-base-uncased`) on 16,000 samples for 3 epochs (batch_size=8, Kaggle P100 GPU). Training parameters: `TrainingArguments` with `eval_strategy='epoch'`, `report_to='none'`.
- **Why**: BERT could potentially capture text context for better results.
- **Result**: After 9 hours of training, we achieved 48% accuracy and ~0.65 F1-score. The model performed at random-guess level, confirming the dataset’s unsuitability for classification.
- **Issues**:
  - Encountered errors like `NameError: TrainingArguments` and `wandb.init()`, resolved with `report_to='none'`.
  - Checkpoints (`./results/checkpoint-6000`) were lost in Kaggle, leading to `HFValidationError`.

## 4. Dataset Analysis
- **What We Did**:
  - **Word Clouds**: Generated word clouds for fake and real news. Common words like `new`, `mr` appeared.
  - **Word Frequencies**: Used `CountVectorizer` to list the top 20 words (fake: `summer`, `wide`; real: `economy`, `member`). Words were distinct but insufficient for contextual separation.
  - **Sentiment Analysis**: Calculated sentiment using `TextBlob` (fake: 0.094, real: 0.093). Scores were nearly identical, indicating no emotional distinction.
  - **Data Cleaning**: Removed low-confidence samples, reducing the dataset to 5,721 samples.
- **Why**: To diagnose why the dataset failed to support classification.
- **Result**: The dataset had noisy labels and contextual similarity. Texts were not distinct enough for fake/real separation.

## 5. Sample Texts
- **Fake News Examples**:
  - `where free section small present stage couple memory bag would real protect page notice...`
  - `or onto strategy first camera stage really almost beautiful whole land thus care...`
  - `executive reflect this family hard drive summer author direction source option help...`
- **Real News Examples**:
  - `ready best rich computer choose middle center expert several rich of nearly voice...`
  - `free themselves keep bill final inside all federal popular serious claim nearly...`
  - `seat reach TV sometimes population treatment ability until outside card case several...`
- **Comment**: Texts share similar vocabulary, making differentiation challenging.

## 6. Lessons Learned
- **Data Quality**: The quality of labels and text distinctiveness is critical for fake news detection.
- **Model Limitations**: TF-IDF and BERT require high-quality data to perform well.
- **Analysis Techniques**: Word clouds, frequency analysis, and sentiment analysis are effective for dataset diagnosis.
- **Kaggle Experience**: Learned to manage checkpoints and handle Kernel timeouts during long training sessions.

## 7. Why We Failed
- **Noisy Labels**: Fake/real labels may be incorrect or inconsistent.
- **Contextual Similarity**: Texts cover similar topics with overlapping vocabulary (e.g., `new`, `mr`).
- **Low Vocabulary Diversity**: TF-IDF produced only 870 features, indicating a limited dataset.

## 8. Conclusion and Closure
- This dataset was unsuitable for fake news detection. The 9-hour BERT training, TF-IDF models, and data analyses confirmed its limitations.
- **Decision**: We are closing this project. A new dataset (e.g., LIAR) would be more viable, but we’re stopping here.
- **Acknowledgment**: This process provided valuable experience in data analysis, machine learning, and deep learning. 

**Date**: June 4, 2025